In [ ]:
#1
%matplotlib inline
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from consts import sensor_day_seconds as day_second
from consts import sensor_idd as idd
from consts import sensor_idds_uniq as idds_uniq
from consts import sensor_idds_uniq_sorted_sa as idds_uniq_sorted_sa
# from consts import sensor_idds_uniq_sorted as idds_uniq_sorted
from consts import sensor_a_temperature as a_temperature
from consts import sensor_s_temperature as s_temperature
from consts import sensor_unix_column as unix_column
from consts import sensor_idd as idd
epoch_length = 30
import multiprocessing as mp
from multiprocessing import Pool
import time

## !!! 在整个流程前,加入Drop NaN

In [2]:
#2
input_file = open('3_dict_list_sa.pkl','rb')
input_list = pickle.load(input_file)
input_file.close()

In [3]:
#3
# 将epoch epoch_len epoch_time 都flat化
len_list = []
time_stamp_list = []
epoch_list = []
for start_time in input_list:
    # print(len(start_time[0]))
    for dict in start_time[0]:
        len_list.append(len(dict))
        epoch_list.append(dict)
    start_point = start_time[1]
    time_stamps = np.arange(start_point,start_point+day_second+epoch_length,epoch_length)
    time_stamp_list.extend(time_stamps[:-1])
len_array = np.array(len_list)
time_stamp_array = np.array(time_stamp_list) # 时间序列
epoch_array = np.array(epoch_list)

ipykernel_launcher:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
# 修改为多线程化读入

In [4]:
#4
def read_sub_df(start_point):
    time_stamps = np.arange(start_point,start_point+day_second+epoch_length,epoch_length)
    sub_df_list = []
    df = pd.read_hdf('day2_sa.h5',str(start_point))
    for i in range(len(time_stamps)-1):
        sub_df = df[(df[unix_column] > time_stamps[i]) & (df[unix_column] <= time_stamps[i+1])]
        sub_df_list.append(sub_df)
    print("Done \t {}".format(start_point))
    return (start_point,sub_df_list)

In [5]:
#5
start_list = np.arange(1162393768,1178747998+day_second,day_second)
manager = mp.Manager()
# aggreateData = manager.dict()
p = Pool(16)
tmp_result = []

for start_point in start_list:
    result = p.apply_async(func=read_sub_df,args=(start_point,))
    tmp_result.append(result)
p.close()
p.join()


Done 	 1162825768
Done 	 1162480168
Done 	 1162566568
Done 	 1162652968
Done 	 1162739368
Done 	 1162912168
Done 	 1163084968
Done 	 1162393768
Done 	 1163171368
Done 	 1162998568
Done 	 1163430568
Done 	 1163257768
Done 	 1163689768
Done 	 1163344168
Done 	 1163603368
Done 	 1163516968
Done 	 1163862568
Done 	 1163776168
Done 	 1163948968
Done 	 1164035368
Done 	 1164121768
Done 	 1164208168
Done 	 1164294568
Done 	 1164380968
Done 	 1164467368
Done 	 1164553768
Done 	 1164640168
Done 	 1164726568
Done 	 1164812968
Done 	 1164899368
Done 	 1164985768
Done 	 1165072168
Done 	 1165158568
Done 	 1165244968
Done 	 1165331368
Done 	 1165417768
Done 	 1165504168
Done 	 1165590568
Done 	 1165676968
Done 	 1165763368
Done 	 1165849768
Done 	 1165936168
Done 	 1166022568
Done 	 1166108968
Done 	 1166195368
Done 	 1166281768
Done 	 1166368168
Done 	 1166454568
Done 	 1166540968
Done 	 1166627368
Done 	 1166713768
Done 	 1166800168
Done 	 1166886568
Done 	 1166972968
Done 	 1167059368
Done 	 116

In [6]:
#6
origin_time = start_list[0]

results = []
for i in start_list:
    results.append([])

for item in tmp_result:
    temp = item.get()
    results[int((temp[0]-origin_time)/86400)].extend(temp[1])

ddf_list = []
for item in results:
    ddf_list.extend(item)

In [ ]:
#7
index1 = np.where(len_array > 40)[0]
maybe_array = epoch_array[index1]

In [ ]:
print(len(maybe_array))

In [ ]:
#8
# init maybe_dict 
maybe_dict = {}
for i in idds_uniq:
    maybe_dict[i] = 0


for item in maybe_array:
    uniq_item = np.unique(item)
    for j in uniq_item:
        maybe_dict[j] += 1
print(maybe_dict)

In [ ]:
#9
maybe_dict_sorted = sorted(maybe_dict.items(),key=lambda kv:(kv[1],kv[0]),reverse=True)
maybe_dict_sorted = np.array(maybe_dict_sorted)

In [ ]:
sub_sensor_list = maybe_dict_sorted.T[0]
print(sub_sensor_list)

In [7]:
#10
# 测试sensor数量为多少合适
def get_sub_index(sensor_count):
    sub_sensor_list = idds_uniq_sorted_sa[:sensor_count]
    sub_index = []
    for i, item in enumerate(epoch_array):
        uniq_item = np.unique(item)
        diff = np.setdiff1d(sub_sensor_list,uniq_item)
        if len(diff) == 0:
            sub_index.append(i)
    return len(sub_index),sub_index

In [8]:
#11 多进程版 
start_time = time.time()
def multi_get_sub_index(i):
    return get_sub_index(i)

sensor_count_list = np.linspace(10,50,41,dtype=int)

manager = mp.Manager()
# aggreateData = manager.dict()
p = Pool(16)
tmp_result = []

for i in sensor_count_list:
    result = p.apply_async(func=multi_get_sub_index,args=(i,))
    tmp_result.append(result)
p.close()
p.join()
print(time.time()-start_time)

88.09694504737854


In [9]:
#11-1
sub_index_list = []
for item in tmp_result:
    sub_index_list.append(item.get())
sub_index_array = np.array(sub_index_list)

ipykernel_launcher:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [10]:
#12
def get_length_count(window):
    for item in sub_index_array:
        zeros = np.zeros_like(epoch_array)
        zeros[item[1]] = 1
        sum = 0
        for j in range(len(zeros)):
            tmp = zeros[j:j+window]
            if 0 not in tmp:
                sum += 1
        print(sum)

In [11]:
# 12-1 DUMP
output_file = open('sub_index_array_drop_na_sa.pkl','wb')
pickle.dump(sub_index_array,output_file)
output_file.close()

In [ ]:
# 12-2 LOAD
input_file = open('sub_index_array_drop_na_sa.pkl','rb')
sub_index_array = pickle.load(input_file)
input_file.close()

In [12]:
#13
def get_data_matrixs(sensor_count,window,sub_index_array = None):
    print(idds_uniq_sorted_sa[:sensor_count])
    tmp = sub_index_array[sensor_count-10][1]
    zeros = np.zeros_like(epoch_array)
    zeros[tmp] = 1

    sum_list = []
    for i in range(len(zeros)):
        tmp1 = zeros[i:i+window]
        if 0 not in tmp1:
            sum_list.append(i)
            
    return sum_list
indices = get_data_matrixs(20,7,sub_index_array)
# print(len(indices))

[121, 35, 14, 26, 46, 8, 23, 34, 17, 33, 75, 40, 72, 73, 89, 97, 70, 19, 49, 79]


In [17]:
#14
# 丢弃多余帧
def drop_useless_sensor(df,sensor_list):
    # print(sensor_list)
    for i in df[idd].values:
        if i not in sensor_list:
            df = df[df[idd] != i]
    # print(len(df))
    if len(df) < len(sensor_list):
        return None
    return df
# t = drop_useless_sensor(ddf_list[indices[3]],idds_uniq_sorted[:19])
# print(len(t))

In [15]:
# before 15
for i, df in enumerate(ddf_list):
    with open('./data_sa/{}.pkl'.format(i),'wb') as f:
        pickle.dump(df,f)

In [18]:
#15 multi_processing
def multi_convert_df_to_dm(indice_num,sensor_list,window=7):
    indices = get_data_matrixs(len(sensor_list),window,sub_index_array)
    print('indices length:{}'.format(len(indices)))
    print(sensor_list)
    print(sorted(sensor_list))
    print(len(sensor_list))
    indice = indices[indice_num]
    df_list = []
    for i in range(window):
        with open('./data_sa/{}.pkl'.format(indice+i),'rb') as f:
            sub_df = pickle.load(f)
        df_list.append(sub_df)
    return_list = []
    for df in df_list:
        df = df.drop_duplicates(idd)
        df = drop_useless_sensor(df,sensor_list)
        if type(df) == type(None):
            break
        df = df.sort_values(by=idd)
        flat_matrix = np.zeros(100)
        # indice_dict = {63:10,23:14,17:15,47:20,7:21,27:22,26:24,14:25,121:26,46:27,8:28,111:32,75:33,57:34,59:35,40:36,34:37,61:41,71:42,65:43,82:45,62:46,76:48,51:51,11:52,70:53,49:54,60:57,84:60,81:61,55:62,72:63,73:64,35:66,31:67,69:68,79:69,103:70,80:71,96:72,97:75,100:76,33:77,19:78,93:79,109:80,87:81,104:84,89:91,88:92,98:94}
        return_list.append(df[s_temperature].values)
    return np.array(return_list).T
res_array = multi_convert_df_to_dm(111,idds_uniq_sorted_sa[:21],7)
print(res_array)
print(res_array.shape)

[121, 35, 14, 26, 46, 8, 23, 34, 17, 33, 75, 40, 72, 73, 89, 97, 70, 19, 49, 79]
indices length:271127
[121, 35, 14, 26, 46, 8, 23, 34, 17, 33, 75, 40, 72, 73, 89, 97, 70, 19, 49, 79]
[8, 14, 17, 19, 23, 26, 33, 34, 35, 40, 46, 49, 70, 72, 73, 75, 79, 89, 97, 121]
20
[[-0.9   -0.9   -0.463 -0.65  -0.838 -1.212 -1.087]
 [-0.275  1.413  0.225 -0.338 -0.338 -0.275 -0.15 ]
 [-0.212 -0.025 -0.087 -0.025 -0.212  0.037 -0.025]
 [ 0.662  0.662  0.725  0.412  0.475  0.6    0.662]
 [ 0.85   0.787  0.912  0.787  0.975 -0.588  0.787]
 [   nan    nan    nan    nan    nan    nan    nan]
 [-1.212 -0.4   -0.525 -0.713 -0.775 -0.713 -0.838]
 [-0.463 -0.525 -0.463 -0.463 -0.4   -0.463 -0.463]
 [ 0.912  1.1    1.288  1.413  1.725  1.225  1.225]
 [ 3.038  3.163  3.35   3.225  3.1    3.1    3.225]
 [ 0.475  0.662  1.163  0.85   0.6    0.537  0.287]
 [-1.587 -1.525 -1.587 -1.275 -1.712 -1.275 -1.525]
 [-0.525 -0.65  -0.463 -0.4   -0.525 -0.4   -0.463]
 [ 0.1    0.037  0.1    0.1    0.163 -0.15   0.1  ]
 [-1

In [22]:
ddf_list[indices[111]]

,Station ID,Year,Month,Day,Hour,Minute,Second,Time since the epoch [s],Sequence Number,Ambient Temperature,Surface Temperature,Relative Humidity [%]
80575,87,2006.0,12.0,21.0,18.0,10.0,35.0,1.166721e+09,45484.0,1.90,-0.650,77.192
134300,17,2006.0,12.0,21.0,18.0,10.0,33.0,1.166721e+09,9032.0,1.66,-0.212,73.186
113305,33,2006.0,12.0,21.0,18.0,10.0,34.0,1.166721e+09,608.0,1.71,-1.212,74.439
137,49,2006.0,12.0,21.0,18.0,10.0,36.0,1.166721e+09,509.0,2.14,-1.587,76.200
137392,121,2006.0,12.0,21.0,18.0,10.0,34.0,1.166721e+09,59262.0,1.74,0.537,75.749
123825,14,2006.0,12.0,21.0,18.0,10.0,35.0,1.166721e+09,2666.0,1.97,-0.275,72.747
83160,89,2006.0,12.0,21.0,18.0,10.0,53.0,1.166721e+09,18114.0,1.48,NaN,78.179
134389,34,2006.0,12.0,21.0,18.0,10.0,54.0,1.166721e+09,12460.0,1.52,-0.463,72.102
75788,49,2006.0,12.0,21.0,18.0,10.0,53.0,1.166721e+09,485.0,1.81,-1.962,74.510
123321,23,2006.0,12.0,21.0,18.0,10.0,49.0,1.166721e+09,23774.0,1.60,0.850,75.229


In [21]:
pd.options.display.max_rows=None

In [ ]:
# obsolete
# 15
def convert_df_to_dm(ddf_list,indice_num,sensor_list,window=7):
    indices = get_data_matrixs(len(sensor_list),window,sub_index_array)
    indice = indices[indice_num]
    # print(sensor_list)
#     print(len(sensor_list))
    df_list = []
    for i in range(window):
        df_list.append(ddf_list[indice+i])
#     df = ddf_list[indice]
    return_list = []
    for df in df_list:
        df = df.drop_duplicates(idd)
#         print(indice,type(df))
#         print(len(df))
        df = drop_useless_sensor(df,sensor_list)
        if type(df) == type(None):
            break
        df = df.sort_values(by=idd)
        flat_matrix = np.zeros(100)
        # indice_dict = {63:10,23:14,17:15,47:20,7:21,27:22,26:24,14:25,121:26,46:27,8:28,111:32,75:33,57:34,59:35,40:36,34:37,61:41,71:42,65:43,82:45,62:46,76:48,51:51,11:52,70:53,49:54,60:57,84:60,81:61,55:62,72:63,73:64,35:66,31:67,69:68,79:69,103:70,80:71,96:72,97:75,100:76,33:77,19:78,93:79,109:80,87:81,104:84,89:91,88:92,98:94}
        return_list.append(df[s_temperature].values)
#         for i,indice in enumerate(df[idd].values):
#             flat_matrix[indice_dict[indice]] = df[s_temperature].values[i]
    return np.array(return_list).T
    

res_array = convert_df_to_dm(ddf_list,114,idds_uniq_sorted[:20],7)
print(res_array)
print(res_array.shape)

In [ ]:
#16多线程
array_list2 = []
manager = mp.Manager()

p = Pool(16)

start_time = time.time()
def multi_get_sub_index(i):
    return get_sub_index(i)

tmp_result = []

for i in indices:
    result = p.apply_async(func=multi_convert_df_to_dm,args=(i,idds_uniq_sorted[:19],7,))
    tmp_result.append(result)
p.close()
p.join()

for item in tmp_result:
    array_list2.append(item.get())

print(time.time()-start_time)

In [ ]:
#before 17-4
sum = 0
new_indices = []
for i,item in enumerate(array_list2):
    if np.isnan(item).any():
        print(i)
#         sum+=1
    else:
        new_indices.append(i)

In [ ]:
time_stamp_array

In [ ]:
#17-1
with open('4_tensor.pkl','wb') as f:
    pickle.dump(array_list2,f)

In [ ]:
#17-2
with open('4_time_stamp_array.pkl','wb') as f:
    pickle.dump(time_stamp_array,f)

In [ ]:
#17-3
with open('4_indices_na.pkl','wb') as f:
    pickle.dump(indices,f)

In [ ]:
#17-4
with open('4_indices_no_na.pkl','wb') as f:
    pickle.dump(new_indices,f)